In [5]:
!pip install streamlit

In [67]:
import csv
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# df = pd.read_csv("datacsv.csv")

# Fixing random state for reproducibility
np.random.seed(19680801)

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_excel('/content/drive/My Drive/datacsvBreak1.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
%%writefile main.py

import streamlit as st
import pandas as pd
import numpy as np
import csv
import random
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM

# Чтение и сохранение содержимого Excel файла
read_file = pd.read_excel("/content/drive/My Drive/datacsvBreak1.xlsx")
read_file.to_csv("datacsvBreak1.csv", index=None, header=True)

# Чтение CSV файла и преобразование в объект DataFrame
df = pd.read_csv("datacsvBreak1.csv")
st.write(df)

# Генерация дополнительных данных
num = 1
listType = ["БП", "УМ"]
data_names = ["№", "Тип устройства", "Отказ", "Время", "Температура уст-ва", "Температура среды", "Категория", "Потребляемая мощность"]
listD = [data_names]

for i in range(12000):
    listElem = [num, random.choice(listType), 0, 8760, random.randint(5, 70), random.randint(-25, 38), random.choice([1, 2]), random.randint(1, 250)]
    listD.append(listElem)
    num += 1

# Сохранение сгенерированных данных в CSV файл
with open('datacsvAll.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(listD)

# Объединение данных из обоих CSV файлов
dfA = pd.concat(map(pd.read_csv, ['datacsvBreak1.csv', 'datacsvAll.csv']), ignore_index=True)

# Подготовка данных для обучения модели
X = dfA[["Температура уст-ва", "Температура среды", "Категория", "Потребляемая мощность"]]
y = dfA["Отказ"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

X_train = np.expand_dims(X_train.values, axis=2)
X_test = np.expand_dims(X_test.values, axis=2)

# Обучение модели
model = tf.keras.Sequential()
model.add(LSTM(units=32, input_shape=(4, 1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, validation_split=0.2, epochs=5, batch_size=32, verbose=1)

# Ввод значений через Streamlit
n1 = st.number_input("Температура уст-ва", key="tu")  # элемент ввод значения
n2 = st.number_input("Температура среды", key="ts")   # элемент ввод значения
n3 = st.number_input("Потребляемая мощность", key="power")  # элемент ввод значения
# n4 = st.selectbox('Категория ремонтопригодности?', [1, 2])  # выбор категории

if st.button('Предсказание!'):
    # Подготовка входных данных для предсказания
    input_array = np.array([[n1, n2, n3]])  # Входные данные должны содержать все необходимые параметры
    input_array = input_array.reshape((1, 3))  # Изменение формы на (1, 4) для одной записи

    # Получение предсказания
    prediction = model.predict(input_array)
    st.write("Предсказание: ", float(prediction[0][0]))  # Вывод предсказания

Overwriting main.py


In [73]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 serveo.net

Forwarding HTTP traffic from https://c7273993fb354228c94cb53b20bf72ef.serveo.net
HTTP request from 79.139.145.143 to https://c7273993fb354228c94cb53b20bf72ef.serveo.net/
HTTP request from 79.139.145.143 to https://c7273993fb354228c94cb53b20bf72ef.serveo.net/static/media/SourceSansPro-Regular.DZLUzqI4.woff2
HTTP request from 79.139.145.143 to https://c7273993fb354228c94cb53b20bf72ef.serveo.net/static/media/SourceSansPro-SemiBold.sKQIyTMz.woff2
HTTP request from 79.139.145.143 to https://c7273993fb354228c94cb53b20bf72ef.serveo.net/static/media/SourceSansPro-Bold.-6c9oR8J.woff2
HTTP request from 79.139.145.143 to https://c7273993fb354228c94cb53b20bf72ef.serveo.net/static/js/index.B-cSXLfy.js
HTTP request from 79.139.145.143 to https://c7273993fb354228c94cb53b20bf72ef.serveo.net/static/css/index.BOl9eq08.css
HTTP request from 79.139.145.143 to https://c7273993fb354228c94cb53b20bf72ef.serveo.net/favicon.png
HTTP request from 79.139.145.143 to https://c7273993fb354228c94cb53b20bf72ef.serveo.